In [1]:
import pandas as pd
from ast import literal_eval
import numpy as np

In [4]:
dataset = 'boardgamegeek'

import os
datapath = os.path.join('..', 'data', dataset)

os.makedirs(datapath, exist_ok=True)
starting_data_path = os.path.join('..', 'data', 'boardgames_archive')
info_items = pd.read_csv(os.path.join(starting_data_path, 'games_detailed_info.csv'), index_col=0) 
info_items.replace(np.nan, '[]', inplace=True)
info_items.to_csv(os.path.join(datapath,'info.csv'), index=False)

C:\Users\petru\AppData\Local\Temp\ipykernel_24344\242640057.py:8: DtypeWarning: Columns (29,48) have mixed types. Specify dtype option on import or set low_memory=False.
  info_items = pd.read_csv(os.path.join(starting_data_path, 'games_detailed_info.csv'), index_col=0)


In [5]:
info_items = pd.read_csv(os.path.join(datapath,'info.csv'), converters={'alternate': literal_eval,\
                                                'boardgamecategory': literal_eval,\
                                                'boardgamemechanic': literal_eval,\
                                                'boardgamefamily': literal_eval,\
                                                'boardgameexpansion': literal_eval,\
                                                'boardgameimplementation': literal_eval,\
                                                'boardgamedesigner': literal_eval,\
                                                'boardgameartist': literal_eval,\
                                                'boardgamepublisher': literal_eval})

C:\Users\petru\AppData\Local\Temp\ipykernel_24344\1908206490.py:1: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  info_items = pd.read_csv(os.path.join(datapath,'info.csv'), converters={'alternate': literal_eval,\


In [22]:
len(info_items)

21631

In [ ]:
#import urllib.request
#import swifter
#
#altready_down = set([x.replace('.jpg', '') for x in os.listdir(os.path.join(starting_data_path, 'images'))])
#
#def download_images(row):
#    id_ = row['id']
#    if id_ in altready_down:
#        return
#    else:
#        if 'https' in row['image']:
#            urllib.request.urlretrieve(row['image'], os.path.join(starting_data_path, 'images', f"{id_}.jpg"))
#        else:
#            return
#
##os.makedirs(os.path.join(starting_data_path, 'images'))
#info_items.swifter.apply(download_images, axis=1)

In [ ]:
rev = pd.read_csv(os.path.join(starting_data_path,'bgg-19m-reviews.csv'), index_col=0)

In [ ]:
rev['rating'].describe()

In [ ]:
168733

In [ ]:
agg_df = rev.groupby('user').agg({
    'ID': list
})
agg_df = agg_df.reset_index()
agg_df['count'] = agg_df['ID'].apply(len)
agg_df = agg_df[agg_df['count']>10]

agg_df = agg_df[agg_df['count']<100]
core_10_user = set(agg_df['user'].tolist())

rev = rev[rev['user'].isin(core_10_user)]

In [ ]:
from itertools import chain

core_items = set(chain(*agg_df['ID'].tolist()))

In [ ]:
info_items.columns

In [ ]:
info_items = info_items.loc[:, ['type', 'id', 'thumbnail', 'image', 'primary', 'alternate',
       'description', 'yearpublished', 'minplayers', 'maxplayers',
       'suggested_num_players', 'suggested_playerage',
       'suggested_language_dependence', 'playingtime', 'minplaytime',
       'maxplaytime', 'minage', 'boardgamecategory', 'boardgamemechanic',
       'boardgamefamily', 'boardgameexpansion', 'boardgameimplementation',
       'boardgamedesigner', 'boardgameartist', 'boardgamepublisher',
       'usersrated',]]

In [ ]:
info_items = info_items[info_items['id'].isin(core_items)]

In [ ]:
#.to_csv("games.txt", sep=";", index=False)
file_name = os.path.join(datapath,'boardgamegeek.txt')
with open(file_name, 'w') as f:
      for _, row in info_items.loc[:, ['id', 'description']].iterrows():
        f.write(f"{row['id']};;{row['description']}\n")

In [ ]:
rev['binary_like'] = rev['rating'].apply(lambda x: 1 if x > 7 else 0)

In [ ]:
rev['binary_like'].describe()

In [ ]:
info_items = info_items.loc[:, ['type', 'id', 'primary', 'alternate',
       'yearpublished', 'minplayers', 'maxplayers',
       'playingtime', 'minplaytime',
       'maxplaytime', 'minage', 'boardgamecategory', 'boardgamemechanic',
       'boardgamefamily', 'boardgameexpansion', 'boardgameimplementation',
       'boardgamedesigner', 'boardgameartist', 'boardgamepublisher']]

In [ ]:
set(info_items['type'].tolist())

In [ ]:
type(info_items.loc[0, 'alternate'])

In [ ]:
import numpy as np
import itertools
import re 
set_prop = set()
for col in info_items.columns:
    set_prop_new = []
    if col == 'id':
        continue
    if type(info_items.loc[0, col])==list:
        set_prop_new = set([re.sub(r'[^\x00-\x7F]+', '', x).strip() if len(x)>3 else "" for x in set(list(itertools.chain(*info_items.loc[:, col].tolist()))) ])
        set_prop = set_prop.union(set(list(filter(lambda x: len(x)>3, set_prop_new))))
        continue
    if str(info_items.loc[0, col]).isnumeric():
        set_prop_new = set(str(x) for x in info_items.loc[:, col].tolist())
        set_prop = set_prop.union(set_prop_new)
        continue
    else:
        set_prop_new = set([re.sub(r'[^\x00-\x7F]+', '', x).strip() if len(x)>3 else "" for x in info_items.loc[:, col].tolist()])
        set_prop = set_prop.union(set(list(filter(lambda x: len(x)>3, set_prop_new))))
        continue

In [ ]:
len(set_prop)

In [ ]:
mapping_dict = {
    i: prop for i, prop in enumerate(set_prop)
}

file_name = os.path.join(datapath, 'mapped_prop.csv')
with open(file_name, 'w') as f:
      for id_, name_ in mapping_dict.items():
        f.write(f"{id_};;{name_}\n")

In [ ]:
mapping_dict_col = {
    i: prop for i, prop in enumerate([x for x in info_items.columns if x != 'id'])
}
file_name = os.path.join(datapath,'mapped_rel.csv')
with open(file_name, 'w') as f:
      for id_, name_ in mapping_dict_col.items():
        f.write(f"{id_};;{name_}\n")

In [ ]:
mapping_dict_col_inv  = {v:k for k, v in mapping_dict_col.items()}
mapping_dict_inv = {v:k for k, v in mapping_dict.items()}
rel = []
for _, row in info_items.iterrows():
    for col in info_items.columns:
        if col == 'id':
            continue
        if type(row[col])==list:
            rel.extend([{
                'id': row['id'],
                'rel': mapping_dict_col_inv[col],
                'prop': mapping_dict_inv.get(x, "")
            } for x in row[col]])
        else:
            rel.append({
                'id': row['id'],
                'rel': mapping_dict_col_inv[col],
                'prop': mapping_dict_inv.get(row[col], "")
            })

In [ ]:
os.makedirs(os.path.join(datapath, 'item-prop'), exist_ok=True)


temp = pd.DataFrame(list(filter(lambda x: len(str(x['prop']))>2, rel)))
from sklearn.model_selection import train_test_split

train, test = train_test_split(temp, test_size=0.2)

train.loc[:, ['id', 'prop', 'rel']].to_csv(os.path.join(datapath, 'item-prop', "train.tsv"), sep='\t', index=False, header=False)
test.loc[:, ['id', 'prop', 'rel']].to_csv(os.path.join(datapath, 'item-prop', "test.tsv"), sep='\t', index=False, header=False)

In [ ]:
for col in info_items.columns:
    set_prop_new = []
    if type(info_items.loc[0, col])==list:
        set_prop_new = set([re.sub(r'[^\x00-\x7F]+', '', x).strip() if len(x)>3 else "" for x in set(list(itertools.chain(*info_items.loc[:, col].tolist()))) ])
        set_prop = set_prop.union(set(list(filter(lambda x: len(x)>3, set_prop_new))))
        continue
    if str(info_items.loc[0, col]).isnumeric():
        set_prop_new = set(str(x) for x in info_items.loc[:, col].tolist())
        set_prop = set_prop.union(set_prop_new)
        continue
    else:
        set_prop_new = set([re.sub(r'[^\x00-\x7F]+', '', x).strip() if len(x)>3 else "" for x in info_items.loc[:, col].tolist()])
        set_prop = set_prop.union(set(list(filter(lambda x: len(x)>3, set_prop_new))))
        continue

In [ ]:
df_train = rev.loc[:, ['user', 'ID', 'binary_like']]
rev.head()

In [ ]:
df_train.to_csv(os.path.join(datapath,"interactions.csv"), sep=";", index=False)

In [ ]:
info_items.loc[:, ['id', 'primary']].to_csv(os.path.join(datapath,"mapped_entities.csv"), sep=";", index=False)

In [3]:
df_train = pd.read_csv(os.path.join(datapath,"interactions.csv"), sep=";")

In [16]:
user_ = len(df_train['user'].unique())
items_ = len(df_train['ID'].unique())

print("# users: ", user_)
print("# items: ", items_)
print("# interaction: ", len(df_train))
print("# sparsity: ", (1-len(df_train)/(user_*items_))*100)

# users:  168733
# items:  21772
# interaction:  6279384
# sparsity:  99.82906984428442


In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_train, test_size=0.2)

In [ ]:
os.makedirs(os.path.join(datapath, 'user-item'), exist_ok=True)

In [ ]:
train.to_csv(os.path.join(datapath, 'user-item', 'train.tsv'), sep='\t', index=False, header=False)

In [ ]:
test.to_csv(os.path.join(datapath,'user-item', 'test.tsv'), sep='\t', index=False, header=False)